In [1]:
import numpy as np
import tinygrad 
from tinygrad.extra.datasets import fetch_cifar
import time
from tinygrad.lazy import Device
Device.DEFAULT = "CUDA"

import cv2
start =time.monotonic()
X_train, Y_train,= fetch_cifar(train=True)

In [2]:
from tinygrad.nn import Linear
from tinygrad.nn import optim
from tinygrad.tensor import Tensor

class TinyCIFAR:
    def __init__ (self):
        self.l1 = Linear(3072,512, bias=True)
        self.l2 = Linear(512,512,bias=True)
        self.l3 = Linear(512,512,bias=True)
        self.l4 = Linear(512,256,bias=True)
        self.l5 = Linear(256,10, bias=True)

    def __call__ (self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        x = x.leakyrelu()
        x = self.l3(x)
        x = x.leakyrelu()
        x = self.l4(x)
        x = x.leakyrelu()
        x = self.l5(x)
        return x.log_softmax()
    #here lies the buggers
    #maybe a dense just doesnt learn CIFAR? 
    
net = TinyCIFAR()

In [3]:
print(X_train.shape)
X_trains = X_train.reshape(50000, -1)
print(X_trains.shape)
Tensor.training= True

from tinygrad.nn.optim import SGD, Adam
opt = SGD([net.l1.weight, net.l2.weight, net.l3.weight], lr=3e-3)

def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

print(Y_train.shape)

(50000, 3, 32, 32)
(50000, 3072)


NameError: name 'SGD' is not defined

In [ ]:




BS=512

#Train the modelieren

for step in range(5000):
    samp=np.random.randint(0, X_trains.shape[0], BS)
    batch = Tensor(X_trains[0], requires_grad=True)
    labels = Y_train[samp]
    out = net(batch)
    loss = cross_entropy(out, labels)
    #print(loss.numpy())
    opt.zero_grad() #NOTE:I'm unsure why it's giving me t.grad is not None (it was array shaping)
    loss.backward()
    opt.step()
    #accuracy dealings
    pred = np.argmax(out.numpy(), axis=-1)
    acc = (pred == labels).mean()
    # if acc >= 1: #how optimistic of myself
    #     break
    if step % 25 == 0:
        print(f'step{step} | loss: {loss.numpy()} | Accuracy: {acc}')
    if step == 800: 
        if acc < 0.15:
            print('Training but not learning')
            break


end=time.monotonic()
print(f'{end-start:.3}s')



In [ ]:
#Validate model
X_test, Y_test= fetch_cifar(train=False)
print(X_test.shape, Y_test.shape)
av_acc = 0 #loopy fucks up without this
X_tests = X_test.reshape(10000, -1)
print(X_tests.shape)
testamount = 100
for step in range(testamount):
    samp = np.random.randint(0, X_tests.shape[0], size=BS)
    batch = Tensor(X_tests[samp], requires_grad=True)
    labels = Y_test[samp]
    out = net (batch)

    pred = np.argmax(out.numpy(), axis=-1)
    av_acc += (pred == labels).mean()
    # print(out.numpy())
    # #print(np.argmax(out.numpy()))
    # print(Y_test[samp])
    # #print(batch.numpy())

print(f"Test Accuracy: {av_acc / testamount}")